# Computación Científica - Tarea 5
---
Vicente Lizana Estivill  
vlizana@alumnos.inf.utfsm.cl  
201310004-K

---

## Introducción
---

En esta tarea analizaremos una _Ecuación Diferencial Parcial_ o _PDE_, la cual corresponde a una ecuación de onda que se propaga en dos dimensiones y a través del tiempo. La idea es llegar a un algoritmo que permita trabajar con los dos dominios adyacentes que propone el ejercicio, los cuales se encuentran mallados en diferentes sistemas coordenados.

Para tiempos altos o mallas finas el cálculo de la solución puede tomar mucho tiempo, es por esto que para hacer más eficientes las visualizaciones, los _widgets_ tienen un _checkbox_ marcado con la palabra **calc**, en caso de que uno necesite unicamente mover el gráfico sin recalcular la solución se recomienda desmarcar esta casilla.

---

## Desarrollo y Análisis de Resultados
---

### Bibliotecas
---

In [1]:
import numpy as np
import ipywidgets as wdg
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

---
### Introducción

Consideremos la siguiente ecuación diferencial parcial, definida en un dominio 
$(x,y) \in \Omega \subset \mathbb{R}^2$ y $0 < t < T_{\max}$:
$$
\begin{align}
\\
 u_{tt}(x,y,t) & = c^2\Delta u(x,y,t), &\quad \text{ para } (x,y) \in \Omega \\
 u(x,y,0) & = u_0(x,y), \quad&\quad \text{ para } (x,y) \in \Omega \\
 u_t(x,y,0) & = u_1(x,y), \quad&\quad \text{ para } (x,y) \in \Omega \\
 u(\partial\Omega ) &= 0 \quad&\quad t > 0.
\end{align}
$$

y la región $\Omega$ tiene la siguiente forma:

<img src="grilla.png" style="float:center;height:500px">

Como se puede apreciar, $\Omega$ es la fusión de una malla cartesiana $\Omega_1$ (Puntos <font color='red'>rojo</font>) con una malla polar $\Omega_2$ (Puntos <font color='blue'>azul</font>) por lo que $\Omega = \Omega_1 \cup \Omega_2$. 

Notar que los puntos en <font color='Purple'>púrpura</font> corresponden a $\displaystyle \theta=\frac{-\pi}{2}$, los puntos  <font color='Goldenrod'>amarillo</font> corresponden a  $ \displaystyle \theta=\frac{\pi}{2}$, y el cuadrado <font color='green'>verde</font> a $r=0$, puntos que seran tratados en las preguntas posteriores.

---
### Pregunta 1

Escriba un esquema explícito para la malla cartesiana con dominio $ (x,y) \in [0,\beta]$x$[-\alpha,\alpha], \alpha=\beta=3$ y resuelva la ecuación de ondas, colocando en un *widget* el tiempo $t$, la velocidad $c$ y los parámetros $\Delta t$ y $\Delta x$. Por simplicidad puede usar $\Delta x = \Delta y$, cuidando de respetar la condición CFL para que sea estable.

Las funciones $u_0$ y $u_1$ vienen dadas por: 
$$
\begin{align}
\\
 u_0(x,y) & = (-\alpha - y)(\beta - x)(\alpha - y)x , \quad \text{ para } (x,y) \in \Omega_1 \\
 u_1(x,y) &= 6\sin\left(2 \pi x\right)\cos\left(\frac{\pi y}{2}\right), \quad \text{ para } (x,y) \in \Omega_1 \\
\end{align}
$$

Calcule el error entre una grilla con $\Delta x = 0.1$ y $\Delta x = 0.01$ (usando la malla mas fina como malla "real"), restando los puntos que tienen en comun ambas grillas. Muestre el error a medida que aumente el parámetro $\Delta t$.

---

Primero discretizamos la ecuación, utilizando _central difference_ en cada término:

$$ \frac{1}{c^2} \frac{\partial^2 u}{\partial t^2} = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \rightarrow \frac{1}{c^2} \frac{u_{i,j,k+1} - 2 u_{i,j,k} + u_{i,j,k-1}}{k^2} = \frac{u_{i+1,j,k} - 2 u_{i,j,k} + u_{i-1,j,k}}{h^2} + \frac{u_{i,j+1,k} - 2 u_{i,j,k} + u_{i,j-1,k}}{h^2} $$

Siendo $i, j$ los índices de la discretización espacial y $k$ el índice temporal. Entonces, como resolveremos iterativamente para cada tiempo, despejamos $u_{i,j,k+1}$, lo que nos deja un esquema explícito para cada punto perteneciente al interior de $\Omega$.

$$ u_{i,j,k+1} = 2 u_{i,j,k} - u_{i,j,k-1} + \frac{c^2 k^2}{h^2} \left( u_{i+1,j,k} + u_{i-1,j,k} + u_{i,j+1,k} + u_{i,j-1,k} - 4 u_{i,j,k} \right) $$

Las condiciones de borde nos permiten obtener resultados para los dos primeros tiempos. Para $t=0$ utilizamos $u(x, y, 0) = u_0(x, y)$, de esta manera:

$$ u_{i,j,0} = u_0(x_i, y_j) = (-\alpha - y_j)(\beta - x_i)(\alpha - y_j)x_i $$

Para $t=1$ podemos utilizar la formula general, pero debemos utilizar la segunda condición de borde para despejar el $u_{i,j,k-1}$. Para esto discretizamos la condición con _central difference_ y despejamos.

$$ \frac{u_{i,j,k+1} - u_{i,j,k-1}}{2k} = u_1(x_i, y_j) \Rightarrow u_{i,j,k-1} = u_{i,j,k+1} - 2ku_1(x_i, y_j) $$

Entonces para el primer paso tenemos:

$$ u_{i,j,k+1} = u_{i,j,k} - 6 k \sin\left(2 \pi x_i\right) \cos\left(\frac{\pi y_j}{2}\right) + \frac{c^2 k^2}{2 h^2} \left( u_{i+1,j,k} + u_{i-1,j,k} + u_{i,j+1,k} + u_{i,j-1,k} - 4 u_{i,j,k} \right) $$

La condición _CFL_ para este caso es:

$$ \frac{2c \cdot \Delta t}{\Delta x} \leqslant 1 $$

---

In [2]:
class PDE1:
    '''
    The reason I made this into a class is because it was the only method I came up with for
    not recalculating the entire PDE when moving the plot around in the interact function.
    '''
    def u_0(self, alpha=3, beta=3):
        return self.mesh_x[1:-1, 1:-1] * \
            (alpha - self.mesh_y[1:-1, 1:-1]) * \
            (beta - self.mesh_x[1:-1, 1:-1]) * \
            (-alpha - self.mesh_y[1:-1, 1:-1])

    def u_1(self):
        return 6 * np.sin(2 * np.pi * self.mesh_x[1:-1, 1:-1]) * \
            np.cos(np.pi * self.mesh_y[1:-1, 1:-1] / 2)
        
    def plot(self, calc=True, elev=20, azim=40, t=1, alpha=3, beta=3, h=0.1, k=0.05, c=1):
        if calc:
            self.solve(t, alpha, beta, h, k, c)
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_title(r'Wave over $\Omega_1$')
        ax.view_init(elev,azim)
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
        ax.plot_surface(self.mesh_x, self.mesh_y, self.z[-1])
        plt.show()
        
    def error(self, t, c):
        # extracting solutions
        self.solve(t, k=0.005, c=c)
        mx, my, mz = self.mesh_x, self.mesh_y, self.z[-1]
        self.solve(t, h=0.01, k=0.005, c=c)
        
        # nd-array masking for extracting common points
        cols = np.zeros(self.mesh_x.shape)
        cols[:,range(0,self.mesh_x.shape[1],10)] = 1
        rows = np.zeros(self.mesh_y.shape)
        rows[range(0,self.mesh_y.shape[0],10),:] = 1
        indexes = np.logical_and(rows, cols)
        error = np.abs(mz - self.z[-1][indexes].reshape(mz.shape))
        
        print("Error para t ={:>5.2f} (k=0.005): {:<12.6}".format(t, np.sum(error)))
        
    def solve(self, t=1, alpha=3, beta=3, h=0.1, k=0.05, c=1):
        # bounds
        x_i = 0
        x_f = beta
        y_i = - alpha
        y_f = alpha

        # times
        n_t = int(t / k)

        # meshgrid
        equi_x = np.linspace(x_i, x_f, int((x_f - x_i)/h))
        equi_y = np.linspace(y_i, y_f, int((y_f - y_i)/h))
        n_x = equi_x.size
        n_y = equi_y.size

        self.mesh_x, self.mesh_y = np.meshgrid(equi_x, equi_y)
        self.z = np.zeros((3, n_y, n_x))

        # initialization
        self.z[0, 1:-1, 1:-1] = self.u_0(alpha, beta)
        if t == 0:
            self.z[-1] = self.z[0]
            return

        # first step
        self.z[1, 1:-1, 1:-1] = \
            self.z[0, 1:-1, 1:-1] + \
            k * self.u_1() + \
            (c ** 2 * k ** 2) / (2 * h ** 2) * (\
                self.z[0, 1:-1, 2:] + \
                self.z[0, 1:-1, :-2] + \
                self.z[0, 2:, 1:-1] + \
                self.z[0, :-2, 1:-1] - \
                4 * self.z[0, 1:-1, 1:-1] \
            )
        if t == k:
            self.z[-1] = self.z[1]
            return

        # general case
        for curr in range(1, n_t-1):
            self.z[2, 1:-1, 1:-1] = \
                2 * self.z[1, 1:-1, 1:-1] - \
                self.z[0, 1:-1, 1:-1] + \
                (c ** 2 * k ** 2) * (\
                    self.z[1, 1:-1, 2:] + \
                    self.z[1, 1:-1, :-2] + \
                    self.z[1, 2:, 1:-1] + \
                    self.z[1, :-2, 1:-1] - \
                    4 * self.z[1, 1:-1, 1:-1] \
                ) / (h ** 2)
            if curr < n_t-2:
                self.z[0] = self.z[1]
                self.z[1] = self.z[2]

---
A continuación una visualilzación de la onda, el código entre _hashtags_ corresponde a la automatización de la condición _CFL_, comentar en caso de requerir más libertad (aparte falla cuando no hay valores posibles para algun _slider_):

In [3]:
elev_slider1 = wdg.IntSlider(min=0, max=180, step=10, value=20, continuous_update=False)
azim_slider1 = wdg.IntSlider(min=0, max=360, step=10, value=40, continuous_update=False)
time1 = wdg.FloatSlider(min=0, max=6, step=0.1, value=0.5, continuous_update=False)
delta_x1 = wdg.FloatSlider(min=0.001, max=0.1, step=0.001, value=0.01, continuous_update=False)
delta_t1 = wdg.FloatSlider(min=0.0005, max=0.05, step=0.0005, value=0.005, continuous_update=False)
c1 = wdg.FloatSlider(min=0.1, max=3, step=0.1, value=1, continuous_update=False)

##############################
# CFL conditions
def uh1(*args):
    try:
        delta_x1.min = 2 * c1.value * delta_t1.value
    except:
        pass
delta_t1.observe(uh1, 'value')
c1.observe(uh1, 'value')
    
def uk1(*args):
    try:
        delta_t1.max = delta_x1.value / (2 * c1.value)
    except:
        pass
delta_x1.observe(uk1, 'value')
c1.observe(uk1, 'value')
    
def uc1(*args):
    try:
        c1.max = delta_x1.value / (2 * delta_t1.value)
    except:
        pass
delta_x1.observe(uc1, 'value')
delta_t1.observe(uc1, 'value')
##############################

pde_1 = PDE1()

_ = wdg.interact(
    pde_1.plot,
    elev = elev_slider1,
    azim = azim_slider1,
    t = time1,
    alpha = wdg.fixed(3),
    beta = wdg.fixed(3),
    h = delta_x1,
    k = delta_t1,
    c = c1
)

A Jupyter Widget

---
Cálculo del Error:

In [4]:
time1_2 = wdg.FloatSlider(min=0.2, max=6, step=0.1, value=0.5, continuous_update=False)
c1_2 = wdg.FloatSlider(min=0.1, max=3, step=0.1, value=1, continuous_update=False)
pde_1_2 = PDE1()

_ = wdg.interact(
    pde_1_2.error,
    t = time1_2,
    c = c1_2
)

A Jupyter Widget

---
### Pregunta 2

- Escriba un esquema explícito para la malla polar con dominio $(r,\theta) \in [0,\alpha]$x$\displaystyle \left[\frac{-\pi}{2},\frac{\pi}{2}\right]$  

- Resuelva la ecuación de ondas, colocando en un *widget* el tiempo $t$, la velocidad $c$ y los parámetro $\Delta t, \Delta r$ y $\Delta  \theta$. Al mover cualquiera de los parámetros $\Delta$, su función debe ajustar el resto de las variables para cumplir con la condición CFL. 

Las funciones $u_0$ y $u_1$ vienen dadas por: 
$$
\begin{align}
\\
 u_0(r,\theta) & = (\alpha - r)\left(\frac{\pi}{2} - \theta\right)\left(\frac{-\pi}{2} - \theta\right)r , \quad \text{ para } (r,\theta) \in \Omega_2 \\
 u_1(r,\theta) &= 6\sin\left(2 \pi r\right)\cos\left(\theta\right), \quad \text{ para } (r,\theta) \in \Omega_2 \\
\end{align}
$$

Dejando $\Delta \theta$ fijo, calcule el error entre una grilla con $\Delta r = 0.1$ y $\Delta r = 0.01$ (usando la malla mas fina como malla "real"), restando los puntos que tienen en comun ambas grillas. Muestre el error a medida que aumente el parámetro $\Delta t$.

---

Antes de discretizar hay que convertir la ecuación al nuevo sistema de coordenadas:

$$ \frac{1}{c^2} \frac{\partial^2 u}{\partial t^2} = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \quad \rightarrow \quad \frac{1}{c^2} \frac{\partial^2 u}{\partial t^2} = \frac{\partial^2 u}{\partial r^2} + \frac{1}{r} \frac{\partial u}{\partial r} + \frac{1}{r^2} \frac{\partial^2 u}{\partial \theta^2} $$

Lo que nos entrega la siguiente discretización:

$$ \frac{1}{c^2} \frac{u_{i,j,k+1} - 2 u_{i,j,k} + u_{i,j,k-1}}{k^2} = \frac{u_{i+1,j,k} - 2 u_{i,j,k} + u_{i-1,j,k}}{h^2_1} + \frac{u_{i+1,j,k} - u_{i-1,j,k}}{2 r_i h_1} + \frac{u_{i,j+1,k} - 2 u_{i,j,k} + u_{i,j-1,k}}{r^2_i h^2_2} $$

Siguiendo el mismo procedimiento anterior tenemos el esquema general:

$$ u_{i,j,k+1} = 2 u_{i,j,k} - u_{i,j,k-1} + c^2 k^2 \left( \frac{u_{i+1,j,k} - 2 u_{i,j,k} + u_{i-1,j,k}}{h^2_1} + \frac{u_{i+1,j,k} - u_{i-1,j,k}}{2 r_i h_1} + \frac{u_{i,j+1,k} - 2 u_{i,j,k} + u_{i,j-1,k}}{r^2_i h^2_2} \right) $$

La primera condición de borde nos entrega:

$$ u_{i,j,0} = u_0(r_i, \theta_j) = (\alpha - r_i) \left( \frac{\pi}{2} - \theta_j \right) \left( -\frac{\pi}{2} - \theta_j \right) r_i $$

Y de la segunda despejamos el término $u_{i,j,-1}$ para utilizarlo en la primera iteración:

$$ \frac{u_{i,j,1} - u_{i,j,-1}}{2k} = u_1(r_i, \theta_j) = 6\sin\left(2 \pi r_i \right)\cos\left(\theta_j\right) $$

Lo que nos deja la primera iteración:

$$ u_{i,j,k+1} = u_{i,j,k} + 6 k \sin\left(2 \pi r_i \right)\cos\left(\theta_j\right) + \frac{c^2 k^2}{2} \left( \frac{u_{i+1,j,k} - 2 u_{i,j,k} + u_{i-1,j,k}}{h^2_1} + \frac{u_{i+1,j,k} - u_{i-1,j,k}}{2 r_i h_1} + \frac{u_{i,j+1,k} - 2 u_{i,j,k} + u_{i,j-1,k}}{r^2_i h^2_2} \right) $$

De las referencias sacamos la condición _CFL_:

$$ 2c\frac{\Delta t}{\Delta r \Delta \theta} \leqslant 1 $$

---

In [5]:
class PDE2:
    def u_0(self, alpha=3):
        return self.mesh_r[1:-1, 1:-1] * \
            (alpha - self.mesh_r[1:-1, 1:-1]) * \
            (np.pi/2 - self.mesh_theta[1:-1, 1:-1]) * \
            (-np.pi/2 - self.mesh_theta[1:-1, 1:-1])

    def u_1(self):
        return 6 * np.sin(2 * np.pi * self.mesh_r[1:-1, 1:-1]) * \
            np.cos(self.mesh_theta[1:-1, 1:-1])
    
    def plot(self, calc=True, elev=20, azim=40, t=1, alpha=3, hr=0.1, ht=0.1, k=0.05, c=1):
        if calc:
            self.solve(t, alpha, hr, ht, k, c)
            
        plot_mesh_x = self.mesh_r * np.cos(self.mesh_theta)
        plot_mesh_y = self.mesh_r * np.sin(self.mesh_theta)
        
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_title(r'Wave over $\Omega_2$')
        ax.view_init(elev,azim)
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
        ax.plot_surface(plot_mesh_x, plot_mesh_y, self.z[-1])
        plt.show()
        
    def error(self, t, c):
        # extracting solutions
        self.solve(t, k=0.005, c=c)
        mr, mt, mz = self.mesh_r, self.mesh_theta, self.z[-1]
        self.solve(t, hr=0.01, k=0.005, c=c)
        
        error = np.abs(mz - self.z[-1][:,range(0,self.mesh_r.shape[1],10)].reshape(mz.shape))
        print("Error para t ={:>5.2f} (k=0.005): {:<12.6}".format(t, np.sum(error)))
        
    def solve(self, t=1, alpha=3, hr=0.1, ht=0.1, k=0.05, c=1):
        # bounds
        r_i = 0
        r_f = alpha
        theta_i = - np.pi/2
        theta_f = np.pi/2

        # times
        n_t = int(t / k)

        # meshgrid
        equi_r = np.linspace(r_i, r_f, int((r_f - r_i)/hr))
        equi_theta = np.linspace(theta_i, theta_f, int((theta_f - theta_i)/ht))
        n_r = equi_r.size
        n_theta = equi_theta.size

        self.mesh_r, self.mesh_theta = np.meshgrid(equi_r, equi_theta)
        self.z = np.zeros((3, n_theta, n_r))

        # initialization
        self.z[0, 1:-1, 1:-1] = self.u_0(alpha)
        if t == 0:
            self.z[-1] = self.z[0]
            return

        # first step
        self.z[0, 1:-1, 1:-1] + \
            k * self.u_1() + \
            (c ** 2 * k ** 2) / 2 * (\
                (self.z[0, 1:-1, 2:] + \
                self.z[0, 1:-1, :-2] - \
                2 * self.z[0, 1:-1, 1:-1]) / (hr ** 2) + \

                (self.z[0, 1:-1, 2:] + \
                self.z[0, 1:-1, :-2]) / (2 * self.mesh_r[1:-1, 1:-1] * hr) + \

                (self.z[0, 2:, 1:-1] + \
                self.z[0, :-2, 1:-1] - \
                2 * self.z[0, 1:-1, 1:-1]) / (self.mesh_r[1:-1, 1:-1] ** 2 * ht ** 2) \
            )
        if t == k:
            self.z[-1] = self.z[1]
            return

        # general case
        for curr in range(1, n_t-1):
            self.z[2, 1:-1, 1:-1] = \
                2 * self.z[1, 1:-1, 1:-1] - \
                self.z[0, 1:-1, 1:-1] + \
                (c ** 2 * k ** 2) * (\
                    (self.z[0, 1:-1, 2:] + \
                    self.z[0, 1:-1, :-2] - \
                    2 * self.z[0, 1:-1, 1:-1]) / (hr ** 2) + \

                    (self.z[0, 1:-1, 2:] + \
                    self.z[0, 1:-1, :-2]) / (2 * self.mesh_r[1:-1, 1:-1] * hr) + \

                    (self.z[0, 2:, 1:-1] + \
                    self.z[0, :-2, 1:-1] - \
                    2 * self.z[0, 1:-1, 1:-1]) / (self.mesh_r[1:-1, 1:-1] ** 2 * ht ** 2) \
                )
            if curr < n_t-2:
                self.z[0] = self.z[1]
                self.z[1] = self.z[2]

---
Visualización de la onda:

In [6]:
elev_slider2 = wdg.IntSlider(min=0, max=180, step=10, value=20, continuous_update=False)
azim_slider2 = wdg.IntSlider(min=0, max=360, step=10, value=40, continuous_update=False)
time2 = wdg.FloatSlider(min=0.2, max=6, step=0.1, value=0.5, continuous_update=False)
delta_r2 = wdg.FloatSlider(min=0.001, max=0.1, step=0.001, value=0.01, continuous_update=False)
delta_theta2 = wdg.FloatSlider(min=0.001, max=0.1, step=0.001, value=0.01, continuous_update=False)
delta_t2 = wdg.FloatSlider(min=0.0005, max=0.05, step=0.0005, value=0.005, continuous_update=False)
c2 = wdg.FloatSlider(min=0.1, max=3, step=0.1, value=1, continuous_update=False)

##############################
# CFL conditions
def uhr2(*args):
    try:
        delta_r2.min = 2 * c2.value * delta_t2.value / delta_theta2.value
    except:
        pass
delta_t2.observe(uhr2, 'value')
delta_theta2.observe(uhr2, 'value')
c2.observe(uhr2, 'value')

def uht2(*args):
    try:
        delta_theta2.min = 2 * c2.value * delta_t2.value / delta_r2.value
    except:
        pass
delta_t2.observe(uht2, 'value')
delta_r2.observe(uht2, 'value')
c2.observe(uht2, 'value')
    
def uk2(*args):
    try:
        delta_t2.max = delta_r2.value * delta_theta2.value / (2 * c2.value)
    except:
        pass
delta_r2.observe(uk2, 'value')
delta_theta2.observe(uk2, 'value')
c2.observe(uk2, 'value')
    
def uc2(*args):
    try:
        c2.max = delta_r2.value * delta_theta2.value / (2 * delta_t2.value)
    except:
        pass
delta_r2.observe(uc2, 'value')
delta_theta2.observe(uc2, 'value')
delta_t2.observe(uc2, 'value')
##############################

pde_2 = PDE2()

_ = wdg.interact(
    pde_2.plot,
    elev = elev_slider2,
    azim = azim_slider2,
    t = time2,
    alpha = wdg.fixed(3),
    hr = delta_r2,
    ht = delta_theta2,
    k = delta_t2,
    c = c2
)

A Jupyter Widget

---
Calculo del error:

In [7]:
time2_2 = wdg.FloatSlider(min=0.2, max=6, step=0.1, value=0.5, continuous_update=False)
c2_2 = wdg.FloatSlider(min=0.1, max=3, step=0.1, value=1, continuous_update=False)
pde_2_2 = PDE2()

_ = wdg.interact(
    pde_2_2.error,
    t = time2_2,
    c = c2_2
)

A Jupyter Widget

---
### Pregunta 3

Fusione las 2 mallas (usando los dominios anteriores), asegurandose de que ambas superficies se muevan al unísono al mover parámetro $t$.

Las funciones $u_0$ y $u_1$ se define como:
$$
u_0 = 0 \in \Omega \\
$$

$$
u_1 =
  \begin{cases}
                                   \displaystyle 6\sin\left(2 \pi x\right)\cos\left(\frac{\pi y}{2}\right) &, (x,y) \in \Omega_1 \\ \\
                                   \displaystyle 6\sin\left(2 \pi r\right)\cos\left(\theta\right) &, (r,\theta) \in \Omega_2 \\  
  \end{cases}
$$

Para la estimación de los puntos que se encuentran en el eje $x = 0$ puede hacer lo siguiente:

 - Para el cuadrado <font color='green'>verde</font>, el cual corresponde a los puntos $u(0,0,t)$ puede usar solamente los puntos de la malla cartesiana.
 - Para los puntos  <font color='Goldenrod'>amarillos</font> y <font color='Purple'>púrpuras</font> las segundas derivadas $u_{xx}$ y $u_{\theta\theta}$ se pueden estimar con diferencias finitas y luego se deben promediar las aproximaciones de *ambas* grillas. Usted debe elegir cual corresponde utilizar para cada color y para cada lado (cartesioano o polar).
 
La estimación con *backward difference* de las segunda derivada corresponde a:
 $$
\frac{u_{0,j}^k - 2u_{-1,j}^k + u_{-2,j}^k}{\Delta t^2} \\
 $$
y *forward difference* se define como:
 $$
 \frac{u_{0,j}^k - 2u_{1,j}^k + u_{2,j}^k}{\Delta t^2} \\
 $$
 
Debe colocar en un *widget* el tiempo $t$, la velocidad $c$ y los parámetro $\Delta t, \Delta x, \Delta y, \Delta r$ y $\Delta  \theta$. Al mover cualquiera de los parámetros *delta*, su función debe ajustar el resto de las variables para cumplir con la condición CFL. Calcule el error entre una grilla con $\Delta x = 0.1$ y $\Delta x = 0.01$ (usando la malla mas fina como malla "real") y con $\Delta r = 0.1$, $\Delta r = 0.01$ restando los puntos que tienen en comun ambas grillas con respecto a sus grillas reales. Muestre el error a medida que aumente el parámetro $\Delta t$.

---

Para unir ámbas mallas primero calculamos por separado cada una, luego calculamos los valores en la frontera de cada una (esto es realizar una estimación por _backward difference_ en la última columna de la solución de la primera malla y promediar con la estimación correspondiente de la primera y última fila de la segunda solución, utilizando _forward_ y _backward difference_ respectivamente (para poder hacer esto notemos que se debe cumplir $\Delta y = \Delta r$)) y posteriormente unir las soluciones.

Como condición _CFL_ simplemente se deben cumplir las dos por separado.

---

In [8]:
class PDE_final:
    '''There\'s probably a better way to do this'''
    def u_1(self, v=False):
        if v:
            return 6 * np.sin(2 * np.pi * self.mesh_x[1:-1, -1]) * \
                np.cos(np.pi * self.mesh_y[1:-1, -1] / 2)
        return 6 * np.sin(2 * np.pi * self.mesh_x[1:-1, 1:-1]) * \
            np.cos(np.pi * self.mesh_y[1:-1, 1:-1] / 2)
        
        
    def u_2(self, v=False, s=0):
        if v:
            return 6 * np.sin(2 * np.pi * self.mesh_r[s, 1:-1]) * \
                np.cos(self.mesh_theta[s, 1:-1])
        return 6 * np.sin(2 * np.pi * self.mesh_r[1:-1, 1:-1]) * \
            np.cos(self.mesh_theta[1:-1, 1:-1])
        
    def plot(self, calc=True, elev=20, azim=40, t=1, alpha=3, beta=3, hx=0.1, hyr=0.1, ht=0.1, k=0.005, c=1):
        if calc:
            self.solve(t, alpha, beta, hx, hyr, ht, k, c)
            
        plot_mesh_x = self.mesh_r * np.cos(self.mesh_theta)
        plot_mesh_y = self.mesh_r * np.sin(self.mesh_theta)
        
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_title(r'Wave over $\Omega$')
        ax.view_init(elev,azim)
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
        ax.plot_surface(self.mesh_x, self.mesh_y, self.z1[-1])
        ax.plot_surface(plot_mesh_x + beta, plot_mesh_y, self.z2[-1])
        plt.show()
        
    def error(self, t, c):
        # extracting solutions
        self.solve(t, c=c)
        mx, my, mz1 = self.mesh_x, self.mesh_y, self.z1[-1]
        mr, mt, mz2 = self.mesh_r, self.mesh_theta, self.z2[-1]
        self.solve(t, hyr=0.01, hx=0.01, c=c)
        
        # nd-array masking for extracting common points
        cols = np.zeros(self.mesh_x.shape)
        cols[:,range(0,self.mesh_x.shape[1],10)] = 1
        rows = np.zeros(self.mesh_y.shape)
        rows[range(0,self.mesh_y.shape[0],10),:] = 1
        indexes = np.logical_and(rows, cols)

        error1 = np.abs(mz1 - self.z1[-1][indexes].reshape(mz1.shape))
        error2 = np.abs(mz2 - self.z2[-1][:,range(0,self.mesh_r.shape[1],10)].reshape(mz2.shape))
        print("Error para t ={:>5.2f} (k=0.005): {:<12.6}".format(t, np.sum(error1) + np.sum(error2)))
        
    def solve(self, t=1, alpha=3, beta=3, hx=0.1, hyr=0.1, ht=0.1, k=0.005, c=1):
        # bounds
        x_i = 0
        x_f = beta
        y_i = - alpha
        y_f = alpha
        r_i = 0
        r_f = alpha
        theta_i = - np.pi/2
        theta_f = np.pi/2

        # times
        n_t = int(t / k)

        # meshgrid
        equi_x = np.linspace(x_i, x_f, int((x_f - x_i)/hx)+1)
        equi_y = np.linspace(y_i, y_f, int((y_f - y_i)/hyr)+1)
        equi_r = np.linspace(r_i, r_f, int((r_f - r_i)/hyr)+1)
        equi_theta = np.linspace(theta_i, theta_f, int((theta_f - theta_i)/ht))
        
        n_x = equi_x.size
        n_y = equi_y.size
        n_r = equi_r.size
        n_theta = equi_theta.size

        # small fix for when frontier sizes do not match
        if n_y % 2 == 0:
            equi_y = np.linspace(y_i, y_f, int((y_f - y_i)/hyr))
            n_y = equi_y.size

        self.mesh_x, self.mesh_y = np.meshgrid(equi_x, equi_y)
        self.z1 = np.zeros((3, n_y, n_x))
        self.mesh_r, self.mesh_theta = np.meshgrid(equi_r, equi_theta)
        self.z2 = np.zeros((3, n_theta, n_r))

        # initialization
        if t == 0:
            return

        # first step
        ## inside region 1
        self.z1[1, 1:-1, 1:-1] = \
            self.z1[0, 1:-1, 1:-1] + \
            k * self.u_1() + \
            (c ** 2 * k ** 2) / 2 * (\
                (self.z1[0, 1:-1, 2:] + \
                self.z1[0, 1:-1, :-2] + \
                2 * self.z1[0, 1:-1, 1:-1]) / (hx ** 2) + \
                                                
                (self.z1[0, 2:, 1:-1] + \
                self.z1[0, :-2, 1:-1] - \
                2 * self.z1[0, 1:-1, 1:-1]) / (hyr ** 2) \
            )
        ## frontier region 1
        self.z1[1, 1:-1, -1] = \
            self.z1[0, 1:-1, -1] + \
            k * self.u_1(v=True) + \
            (c ** 2 * k ** 2) / 2 * (\
                (self.z1[0, 1:-1, -1] + \
                self.z1[0, 1:-1, -3] + \
                2 * self.z1[0, 1:-1, -2]) / (hx ** 2) + \
                                                
                (self.z1[0, 2:, -1] + \
                self.z1[0, :-2, -1] - \
                2 * self.z1[0, 1:-1, -1]) / (hyr ** 2) \
            )
        ## inside region 2
        self.z2[1, 1:-1, 1:-1] + \
            k * self.u_2() + \
            (c ** 2 * k ** 2) / 2 * (\
                (self.z2[0, 1:-1, 2:] + \
                self.z2[0, 1:-1, :-2] - \
                2 * self.z2[0, 1:-1, 1:-1]) / (hyr ** 2) + \

                (self.z2[0, 1:-1, 2:] + \
                self.z2[0, 1:-1, :-2]) / (2 * self.mesh_r[1:-1, 1:-1] * hyr) + \

                (self.z2[0, 2:, 1:-1] + \
                self.z2[0, :-2, 1:-1] - \
                2 * self.z2[0, 1:-1, 1:-1]) / (self.mesh_r[1:-1, 1:-1] ** 2 * ht ** 2) \
            )
        ## frontier region 2
        self.z2[1, 0, 1:-1] + \
            k * self.u_2(v=True, s=0) + \
            (c ** 2 * k ** 2) / 2 * (\
                (self.z2[0, 0, 2:] + \
                self.z2[0, 0, :-2] - \
                2 * self.z2[0, 0, 1:-1]) / (hyr ** 2) + \

                (self.z2[0, 0, 2:] + \
                self.z2[0, 0, :-2]) / (2 * self.mesh_r[0, 1:-1] * hyr) + \

                (self.z2[0, 0, 1:-1] + \
                self.z2[0, 2, 1:-1] - \
                2 * self.z2[0, 1, 1:-1]) / (self.mesh_r[0, 1:-1] ** 2 * ht ** 2) \
            )
        self.z2[1, -1, 1:-1] + \
            k * self.u_2(v=True, s=-1) + \
            (c ** 2 * k ** 2) / 2 * (\
                (self.z2[0, -1, 2:] + \
                self.z2[0, -1, :-2] - \
                2 * self.z2[0, -1, 1:-1]) / (hyr ** 2) + \

                (self.z2[0, -1, 2:] + \
                self.z2[0, -1, :-2]) / (2 * self.mesh_r[-1, 1:-1] * hyr) + \

                (self.z2[0, -1, 1:-1] + \
                self.z2[0, -3, 1:-1] - \
                2 * self.z2[0, -2, 1:-1]) / (self.mesh_r[-1, 1:-1] ** 2 * ht ** 2) \
            )
        ## intersection
        aux = (self.z1[1, :, -1] + np.hstack([ self.z2[1, 0, 1:][::-1], [0], self.z2[1, -1, 1:] ])) /2
        aux[aux.size//2] *= 2
        self.z1[1, :, -1] = aux
        self.z2[1, 0, 1:] = aux[:aux.size//2][::-1]
        self.z2[1, -1, 1:] = aux[(aux.size//2)+1:]
        self.z2[1, :, 0] = aux[aux.size//2]
        ## end condition
        if t == k:
            self.z1[-1] = self.z1[1]
            self.z2[-1] = self.z2[1]
            return

        # general case
        for curr in range(1, n_t-1):
            ## inside region 1
            self.z1[2, 1:-1, 1:-1] = \
                2 * self.z1[1, 1:-1, 1:-1] - \
                self.z1[0, 1:-1, 1:-1] + \
                (c ** 2 * k ** 2) * (\
                    (self.z1[1, 1:-1, 2:] + \
                    self.z1[1, 1:-1, :-2] + \
                    2 * self.z1[1, 1:-1, 1:-1]) / (hx ** 2) + \

                    (self.z1[1, 2:, 1:-1] + \
                    self.z1[1, :-2, 1:-1] - \
                    2 * self.z1[1, 1:-1, 1:-1]) / (hyr ** 2) \
                )
            ## frontier region 1
            self.z1[2, 1:-1, -1] = \
                2 * self.z1[1, 1:-1, -1] - \
                self.z1[0, 1:-1, -1] + \
                (c ** 2 * k ** 2) * (\
                    (self.z1[1, 1:-1, -1] + \
                    self.z1[1, 1:-1, -3] + \
                    2 * self.z1[1, 1:-1, -2]) / (hx ** 2) + \

                    (self.z1[1, 2:, -1] + \
                    self.z1[1, :-2, -1] - \
                    2 * self.z1[1, 1:-1, -1]) / (hyr ** 2) \
                )
            ## inside region 2
            self.z2[2, 1:-1, 1:-1] = \
                2 * self.z2[1, 1:-1, 1:-1] - \
                self.z2[0, 1:-1, 1:-1] + \
                (c ** 2 * k ** 2) * (\
                    (self.z2[1, 1:-1, 2:] + \
                    self.z2[1, 1:-1, :-2] - \
                    2 * self.z2[1, 1:-1, 1:-1]) / (hyr ** 2) + \

                    (self.z2[1, 1:-1, 2:] + \
                    self.z2[1, 1:-1, :-2]) / (2 * self.mesh_r[1:-1, 1:-1] * hyr) + \

                    (self.z2[1, 2:, 1:-1] + \
                    self.z2[1, :-2, 1:-1] - \
                    2 * self.z2[1, 1:-1, 1:-1]) / (self.mesh_r[1:-1, 1:-1] ** 2 * ht ** 2) \
                )
            ## frontier region 2
            self.z2[2, 0, 1:-1] = \
                2 * self.z2[1, 0, 1:-1] - \
                self.z2[0, 0, 1:-1] + \
                (c ** 2 * k ** 2) * (\
                    (self.z2[1, 0, 2:] + \
                    self.z2[1, 0, :-2] - \
                    2 * self.z2[1, 0, 1:-1]) / (hyr ** 2) + \

                    (self.z2[1, 0, 2:] + \
                    self.z2[1, 0, :-2]) / (2 * self.mesh_r[0, 1:-1] * hyr) + \

                    (self.z2[1, 0, 1:-1] + \
                    self.z2[1, 2, 1:-1] - \
                    2 * self.z2[1, 1, 1:-1]) / (self.mesh_r[0, 1:-1] ** 2 * ht ** 2) \
                )
            self.z2[2, -1, 1:-1] = \
                2 * self.z2[1, -1, 1:-1] - \
                self.z2[0, -1, 1:-1] + \
                (c ** 2 * k ** 2) * (\
                    (self.z2[1, -1, 2:] + \
                    self.z2[1, -1, :-2] - \
                    2 * self.z2[1, -1, 1:-1]) / (hyr ** 2) + \

                    (self.z2[1, -1, 2:] + \
                    self.z2[1, -1, :-2]) / (2 * self.mesh_r[-1, 1:-1] * hyr) + \

                    (self.z2[1, -1:, 1:-1] + \
                    self.z2[1, -3, 1:-1] - \
                    2 * self.z2[1, -2, 1:-1]) / (self.mesh_r[-1, 1:-1] ** 2 * ht ** 2) \
                )
            ## intersection
            aux = (self.z1[2, :, -1] + np.hstack([ self.z2[2, 0, 1:][::-1], [0], self.z2[2, -1, 1:] ])) /2
            aux[aux.size//2] *= 2
            self.z1[2, :, -1] = aux
            self.z2[2, 0, 1:] = aux[:aux.size//2][::-1]
            self.z2[2, -1, 1:] = aux[(aux.size//2)+1:]
            self.z2[2, :, 0] = aux[aux.size//2]
            ## next step condition
            if curr < n_t-2:
                self.z1[0] = self.z1[1]
                self.z2[0] = self.z2[1]
                self.z1[1] = self.z1[2]
                self.z2[1] = self.z2[2]

---

Visualización de la Onda:

In [9]:
elev_slider3 = wdg.IntSlider(min=0, max=180, step=10, value=20, continuous_update=False)
azim_slider3 = wdg.IntSlider(min=0, max=360, step=10, value=40, continuous_update=False)
time3 = wdg.FloatSlider(min=0.2, max=6, step=0.1, value=0.5, continuous_update=False)
delta_x3 = wdg.FloatSlider(min=0.001, max=0.1, step=0.001, value=0.1, continuous_update=False)
delta_yr3 = wdg.FloatSlider(min=0.001, max=0.1, step=0.001, value=0.1, continuous_update=False)
delta_theta3 = wdg.FloatSlider(min=0.001, max=0.1, step=0.001, value=0.1, continuous_update=False)
delta_t3 = wdg.FloatSlider(min=0.0005, max=0.05, step=0.0005, value=0.005, continuous_update=False)
c3 = wdg.FloatSlider(min=0.1, max=3, step=0.1, value=1, continuous_update=False)

##############################
# CFL conditions
def uhx3(*args):
    try:
        delta_x3.min = 2 * c3.value * delta_t3.value / delta_yr3.value
    except:
        pass
delta_t3.observe(uhx3, 'value')
delta_yr3.observe(uhx3, 'value')
c3.observe(uhx3, 'value')
    
def uhy3(*args):
    try:
        delta_yr3.min = 2 * c3.value * delta_t3.value / delta_x3.value
    except:
        pass
delta_t3.observe(uhy3, 'value')
delta_x3.observe(uhy3, 'value')
c3.observe(uhy3, 'value')

def uk3_1(*args):
    try:
        delta_t3.max = delta_x3.value * delta_yr3.value / (2 * c3.value)
    except:
        pass
delta_x3.observe(uk3_1, 'value')
delta_yr3.observe(uk3_1, 'value')
c3.observe(uk3_1, 'value')
    
def uc3_1(*args):
    try:
        c3.max = delta_x3.value * delta_yr3.value / (2 * delta_t3.value)
    except:
        pass
delta_x3.observe(uc3_1, 'value')
delta_yr3.observe(uc3_1, 'value')
delta_t3.observe(uc3_1, 'value')
##############################
def uhr3(*args):
    try:
        delta_yr3.min = 2 * c3.value * delta_t3.value / delta_theta3.value
    except:
        pass
delta_t3.observe(uhr3, 'value')
delta_theta3.observe(uhr3, 'value')
c3.observe(uhr3, 'value')

def uht3(*args):
    try:
        delta_theta3.min = 2 * c3.value * delta_t3.value / delta_yr3.value
    except:
        pass
delta_t3.observe(uht3, 'value')
delta_yr3.observe(uht3, 'value')
c3.observe(uht3, 'value')
    
def uk3_2(*args):
    try:
        delta_t3.max = delta_yr3.value * delta_theta3.value / (2 * c3.value)
    except:
        pass
delta_yr3.observe(uk3_2, 'value')
delta_theta3.observe(uk3_2, 'value')
c3.observe(uk3_2, 'value')
    
def uc3_2(*args):
    try:
        c3.max = delta_yr3.value * delta_theta3.value / (2 * delta_t3.value)
    except:
        pass
delta_yr3.observe(uc3_2, 'value')
delta_theta3.observe(uc3_2, 'value')
delta_t3.observe(uc3_2, 'value')
##############################

pde_3 = PDE_final()

_ = wdg.interact(
    pde_3.plot,
    elev = elev_slider3,
    azim = azim_slider3,
    t = time3,
    alpha = wdg.fixed(3),
    beta = wdg.fixed(3),
    hx = delta_x3,
    hyr = delta_yr3,
    ht = delta_theta3,
    k = delta_t3,
    c = c3
)

A Jupyter Widget

---
Error:

In [10]:
time3_2 = wdg.FloatSlider(min=0.2, max=6, step=0.1, value=0.5, continuous_update=False)
c3_2 = wdg.FloatSlider(min=0.1, max=3, step=0.1, value=1, continuous_update=False)
pde_3_2 = PDE_final()

_ = wdg.interact(
    pde_3_2.error,
    t = time3_2,
    c = c3_2
)

A Jupyter Widget

---
## Conclusiones
---

* Una de las mayores conclusiones o más obvias es que computacionalmente hablando es mejor utilizar coordenadas cartesianas, ya que las coordenadas polares tienen el problema de que cerca del origen los puntos están demasiado cerca y hay que dividir por numeros muy cercanos a cero, lo que implica ruido y pérdida de significancia.
* Las condiciones _CFL_ en sistemas de coordenadas curvilíneos no son directos o fáciles de derivar, no parece haber una fórmula general.
* Resolver numéricamente _PDEs_ mediante discretización es una herramienta muy poderosa y flexible, siempre se puede encontrar una forma de que el orden de convergencia con respecto al grosor del mallado sea cuadrático.
* En este trabajo aprendimos a variar la malla de discretización dependiendo de las características del dominio y a unir los resultados para llegar a una solución final.

---

## Referencias
---

[Laplaciano en Coordenadas Polares](http://ramanujan.math.trinity.edu/rdaileda/teach/s12/m3357/lectures/lecture_3_27_2_short.pdf)    
[CFL condition en Coordenadas Polares](https://scicomp.stackexchange.com/questions/20279/how-can-i-solve-wave-equation-for-circular-membrane-in-polar-coordinates/20354#20354)